In [1]:
import pandas as pd
import numpy as np

# Загружаем файл, указывая отсутствие заголовков столбцов, переименовываем столбцы
df = pd.read_csv('C:\Учеба\mg_bot_messages.csv',low_memory=False, header=None, names=["id", "created_at", "updated_at",
                                                                                      "message_id", "chat_id", "message_text",
                                                                                      "channel", "from", "customer_nickname", 
                                                                                      "user_id", "user_name", "customer_chat_id",
                                                                                      "chat_external_id", "crm_customer_id"])


# Добавляем фамилии менеджерам, там где они были не указаны и изменяем 'channel' на 'user'
df=df.replace(['Наиля','Вика', 'Яна', "Екатерина", 'Йолдыз','Карина','Ирина','Настя','Алина'], 
              ['Наиля Зарипова', 'Виктория Кузьмина','Яна Искандерова', "Екатерина Съемщикова", 
               "Йолдыз Гайнуллина","Карина Газарян","Ирина Распивина","Анастасия Андрамонова","Алина Семенова"])
df = df.replace('channel','user')

# Удаляем ненужные столбцы и строки 
df = df.drop(["id","updated_at","message_id","channel","customer_nickname","user_id","customer_chat_id","chat_external_id","crm_customer_id"], axis =1)
df = df[(df["created_at"]>'2021-06')&(df["created_at"]<'2022-01')&(df["from"]!='bot')&(df["user_name"]!='Техническая Поддержка')&(df["user_name"]!='Дмитрий')]

# Изменяем тип столбца
df["created_at"] = pd.to_datetime(df["created_at"])

# Создаем еще один DataFrame для работы
df_1=df

# Сортируем его по id_диологов и дате, заполняем значения Nan ближайшем нижнем значением
df_1 = df_1.sort_values(by = ["chat_id","created_at"])
df_1["user_name"] = df_1["user_name"].bfill()
# display(df_1.head(50))

# Ищем последнии сообщения в каждой паре клиент - менеджер каждом диалоге
last_mes = df_1.drop_duplicates(subset = ["chat_id",'user_name'], keep='last')
# display(last_mes_chat.head(50))

# Ищем первые сообщения клиентов каждой пары клиент - менеджер в каждом диалоге
first_mes = df_1.drop_duplicates(subset = ["chat_id","user_name"])
first_mes = first_mes[(first_mes["from"]!='user')]
# display(first_mes.head(50))

# Ищем вторые сообщения(ответы менеджера) каждой пары клиент - менеджер в каждом диалоге
second_mes = df_1.drop_duplicates(subset = ["chat_id","from","user_name"])
second_mes = second_mes[(second_mes["from"]!='customer')]
# display(second_mes.head(50))

# Ищем первые сообщения клиентов в каждом диалоге
first_mes_chat = df_1.drop_duplicates(subset = "chat_id")
first_mes_chat = first_mes_chat[(first_mes_chat["from"]!='user')]
# display(first_mes_chat.head(50))

# Ищем вторые сообщения(ответы менеджера) в каждом диалоге
second_mes_chat = df_1.drop_duplicates(subset = ["chat_id","from"])
second_mes_chat = second_mes_chat[(second_mes_chat["from"]!='customer')]
# display(second_mes_chat.head(50))

# ВЫЧИСЛЯЕМ СРЕДНЮЮ СКОРОСТЬ ОТВЕТА НА СООБЩЕНИЯ

# Объеденяем два DataFrame по двум общим столбцам
avg_speed_resp_mes = first_mes.merge(second_mes, on=["chat_id","user_name"])

# Создаем новый столбец равный разницы даты первого сообщения и второго, переводим в минуты полученные значения
avg_speed_resp_mes['created_at_z'] = avg_speed_resp_mes['created_at_y'] - avg_speed_resp_mes['created_at_x']
avg_speed_resp_mes['created_at_z'] = avg_speed_resp_mes['created_at_z'].dt.total_seconds().div(60).astype(int)

# Преобразуем столбец с датами в значения с одними месяцами
avg_speed_resp_mes['created_at_y'] = avg_speed_resp_mes['created_at_y'].dt.month

# Вычисляем среднюю скорость ответа на сообщения
avg_speed_resp_mes = avg_speed_resp_mes.groupby(['user_name','created_at_y'])['created_at_z'].agg('median').unstack().fillna(0).astype(int)
# display(avg_speed_resp_mes)

# ВЫЧИСЛЯЕМ СКОРОСТЬ ОТВЕТА НА ПЕРВОЕ СООБЩЕНИЕ В ДИАЛОГЕ

# Объеденяем два DataFrame по двум общим столбцам
speed_first_resp_mes = first_mes_chat.merge(second_mes_chat, on=["chat_id","user_name"])

# Создаем новый столбец равный разницы даты первого сообщения и второго, переводим в минуты полученные значения
speed_first_resp_mes['created_at_z'] = speed_first_resp_mes['created_at_y'] - speed_first_resp_mes['created_at_x']
speed_first_resp_mes['created_at_z'] = speed_first_resp_mes['created_at_z'].dt.total_seconds().div(60).astype(int)

# Преобразуем столбец с датами в значения с одними месяцами
speed_first_resp_mes['created_at_y'] = speed_first_resp_mes['created_at_y'].dt.month

# Вычисляем скорость ответа на первое сообщение в диалоге
speed_first_resp_mes = speed_first_resp_mes.groupby(['user_name','created_at_y'])['created_at_z'].agg('median').unstack().fillna(0).astype(int)
# display(speed_first_resp_mes)

# ВЫЧИСЛЯЕМ СРЕДНЮЮ ДЛИТЕЛЬНОСТЬ ДИАЛОГОВ

# Объеденяем два DataFrame по двум общим столбцам и изменяем название столбцов
avg_duration_diolog  = first_mes.merge(last_mes, on=["chat_id",'user_name'])

# Создаем новый столбец равный разницы даты последнего сообщения и первого, переводим в минуты полученные значения
avg_duration_diolog['created_at_z'] = avg_duration_diolog['created_at_y'] - avg_duration_diolog['created_at_x']
avg_duration_diolog['created_at_z'] = avg_duration_diolog['created_at_z'].dt.total_seconds().div(60).astype(int)

# Преобразуем столбец с датами в значения с одними месяцами
avg_duration_diolog['created_at_y'] = avg_duration_diolog['created_at_y'].dt.month

# Вычисляем среднюю длительность диалога
avg_duration_diolog = avg_duration_diolog.groupby(['user_name','created_at_y'])['created_at_z'].agg('median').unstack().fillna(0).astype(int)
# display(avg_duration_diolog)



# ВЫЧИСЛЯЕМ ОБЩЕЕ КОЛИЧЕСТВО СООБЩЕНИЙ В ДИАЛОГЕ

all_messages = df

# Преобразуем дату,время в месяцы
all_messages['created_at'] = all_messages['created_at'].dt.month

# Производим сортировку по дате и id_диалога
all_messages = all_messages.sort_values(by = ['chat_id','created_at'])

# Производим автозаполнение ячеек Nan предыдущем существующим значением в ячейки
all_messages = all_messages.bfill()

all_messages = all_messages.groupby(['user_name','created_at'])['user_name'].agg('count').unstack().fillna(0).astype(int)

# ВЫЧИСЛЯЕМ КОЛИЧЕСТВО ИСХОДЯЩИХ СООБЩЕНИЙ

sent_messages = df

# Строим таблицу группируя по двум параметрам, переворачиваем один индекс в название столбцов и считаем количество имен по месяцам 
# т.к. они совпадают с количеством исходящих сообщений
sent_messages = sent_messages.groupby(['user_name','created_at'])['user_name'].agg('count').unstack().fillna(0).astype(int)


# ВЫЧИСЛЯЕМ КОЛИЧЕСТВО ВХОДЯЩИХ СООБЩЕНИЙ

incom_messages = all_messages - sent_messages

# Изменяем название столбцов и индексов
sent_messages.index.name = 'Менеджеры'
sent_messages.columns = [["июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],
                         ['Среднее количество исходящих сообщений','Среднее количество исходящих сообщений',
                          'Среднее количество исходящих сообщений','Среднее количество исходящих сообщений',
                          'Среднее количество исходящих сообщений','Среднее количество исходящих сообщений',
                          'Среднее количество исходящих сообщений']]

incom_messages.index.name = 'Менеджеры'
incom_messages.columns = [["июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],
                          ['Среднее количество входящих сообщений',
                           'Среднее количество входящих сообщений','Среднее количество входящих сообщений',
                           'Среднее количество входящих сообщений','Среднее количество входящих сообщений',
                           'Среднее количество входящих сообщений','Среднее количество входящих сообщений']]

all_messages.index.name = 'Менеджеры'
all_messages.columns = [["июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],
                        ['Среднее количество сообщений в диалоге','Среднее количество сообщений в диалоге',
                         'Среднее количество сообщений в диалоге','Среднее количество сообщений в диалоге',
                         'Среднее количество сообщений в диалоге','Среднее количество сообщений в диалоге',
                         'Среднее количество сообщений в диалоге']]

avg_speed_resp_mes.index.name = 'Менеджеры'
avg_speed_resp_mes.columns = [["июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],
                        ['Средняя скорость ответа на сообщения','Средняя скорость ответа на сообщения',
                         'Средняя скорость ответа на сообщения','Средняя скорость ответа на сообщения',
                         'Средняя скорость ответа на сообщения','Средняя скорость ответа на сообщения',
                         'Средняя скорость ответа на сообщения']]

speed_first_resp_mes.index.name = 'Менеджеры'
speed_first_resp_mes.columns = [["июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],
                        ['Cкорость ответа на первое сообщение','Cкорость ответа на первое сообщение',
                         'Cкорость ответа на первое сообщение','Cкорость ответа на первое сообщение',
                         'Cкорость ответа на первое сообщение','Cкорость ответа на первое сообщение',
                         'Cкорость ответа на первое сообщение']]

avg_duration_diolog.index.name = 'Менеджеры'
avg_duration_diolog.columns = [["июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],
                        ['Средняя длительность диологов','Средняя длительность диологов',
                         'Средняя длительность диологов','Средняя длительность диологов',
                         'Средняя длительность диологов','Средняя длительность диологов',
                         'Средняя длительность диологов']]

# Объединяем таблиц

speed_first_resp_mes = speed_first_resp_mes.stack(1)
avg_duration_diolog = avg_duration_diolog.stack(1)
avg_speed_resp_mes = avg_speed_resp_mes.stack(1)
sent_messages = sent_messages.stack(1)
incom_messages = incom_messages.stack(1)
all_messages = all_messages.stack(1)
result = pd.concat([speed_first_resp_mes, avg_speed_resp_mes, avg_duration_diolog, sent_messages, incom_messages, all_messages]).unstack(1)

# Выводим конечный результат на экран
display(result)


август  \
                      Cкорость ответа на первое сообщение   
Менеджеры                                                   
Аделина Шагеева                                        27   
Алина Семенова                                         59   
Анастасия Андрамонова                                  16   
Анна Жадяева                                            8   
Виктория Кузьмина                                      50   
Динара Рахматуллина                                    13   
Екатерина Съемщикова                                   15   
Елена Кабанова                                          0   
Ирина Распивина                                        38   
Йолдыз Гайнуллина                                      10   
Карина Газарян                                          1   
Наиля Зарипова                                         17   
Полина Мельникова                                       5   
Татьяна Кандалина                                       7   
Яна Искандерова                                        23   
Яна Шабарчина                                          17   

                                                             \
                      Среднее количество входящих сообщений   
Менеджеры                                                     
Аделина Шагеева                                        6759   
Алина Семенова                                         3131   
Анастасия Андрамонова                                  2286   
Анна Жадяева                                           5229   
Виктория Кузьмина                                      2666   
Динара Рахматуллина                                    4649   
Екатерина Съемщикова                                   3907   
Елена Кабанова                                            0   
Ирина Распивина                                        4270   
Йолдыз Гайнуллина                                      5237   
Карина Газарян                                         3405   
Наиля Зарипова                                         5789   
Полина Мельникова                                      5190   
Татьяна Кандалина                                      4894   
Яна Искандерова                                        5136   
Яна Шабарчина                                          2413   

                                                              \
                      Среднее количество исходящих сообщений   
Менеджеры                                                      
Аделина Шагеева                                         8534   
Алина Семенова                                          3602   
Анастасия Андрамонова                                   2170   
Анна Жадяева                                            6158   
Виктория Кузьмина                                       2630   
Динара Рахматуллина                                     4684   
Екатерина Съемщикова                                    5273   
Елена Кабанова                                             0   
Ирина Распивина                                         3502   
Йолдыз Гайнуллина                                       6178   
Карина Газарян                                          5004   
Наиля Зарипова                                          7700   
Полина Мельникова                                       7717   
Татьяна Кандалина                                       5497   
Яна Искандерова                                         6705   
Яна Шабарчина                                           3029   

                                                              \
                      Среднее количество сообщений в диалоге   
Менеджеры                                                      
Аделина Шагеева                                        15293   
Алина Семенова                                          6733   
Анастасия Андрамонова                                   4456   
Анна Жадяева                                           11387   
Виктория Кузьмина           